# **Regression Discontinuity Designs**

Following Wooldridge (2010), CH 21.5

In [ ]:
import stata_setup
stata_setup.config("/Applications/Stata/", "mp")

## Sharp Discontinuity Design (SRD)

### **Notation:**
- $y_0, y_1$ : potential outcomes
- $x$ : forcing variable
- $w = 1[x_i \geq c] $: treatment in SRD
- $c$ : cutoff
- $y = y_0(1-w) + y_1w$ : observed outcome
- $\mu_g(x) \equiv E[y_g | x]$ : counterfactual conditional mean of $y_g$
- $\mu_{gc} \equiv \mu_g(c)$ : counterfactual mean at the cutoff
- $m(x) \equiv E[y|x]$ : conditional mean in terms of observables
- $m_g(x) \equiv E[y | x, w = g]$: conditional mean in terms of observables
- $m^+(c) \equiv \lim_{x\downarrow c} m(x)$ : limit from the right of $m(x)$
- $m^-(c) \equiv \lim_{x\uparrow c} m(x)$ : limit from the left of $m(x)$
- $\tau_c \equiv E[y_1-y_0|x=c] = \mu_1(c) - \mu_0(c)$ : **treatment effect at the cutoff**

**Observed variables:** $(y, x, w)$

### **Assumptions:**
- **Continuity:** $\mu_g(x)$ is continuous at $x=c$ for $g=0,1$
- **Ignorability:** $E[y_g|x, w] = E[y_g | x]$ for $g=0,1$
    - holds because $w$ is determined by $x$

- Note that
    1. Overlap assumption fails, so we cannot use propensity score weighting
    2. Regression adjustment could work, but it would rely on potentially unreasonable extrapolations



### **Identification:**

We have that

$$ m(x) = 1[x<c]\mu_0(x) + 1[x\geq c]\mu_1(x)$$

Hence, by continuity, $m^+(c) = \mu_1(c)$ and $m^-(c) = \mu_0(c)$

So that $\tau_c = m^+(c) - m^-(c)$

### **Estimation:**

#### **Local linear regression**

Model of counterfactual outcomes (note that intercept has mean interpretation because we center $x$ at $c$):

$$ y_0 = \mu_{0c} + \beta_0(x-c) + u_0 $$
$$ y_1 = \mu_{1c} + \beta_1(x-c) + u_1 $$

In terms of observables $y, x, w$, we have the following regression model:

$$ y = \mu_{0c} + \tau_c w + \beta_0(x-c) + \delta w  (x-c) + u $$

where $u = wu_1 + (1-w)u_0$

- Run the regresion choosing a small window $c-h < x <c+h$, can add polynomials and their interactions with $w$ for extra flexibility.
- Can also run separate regressions for the subgroups below and above threshold and then $\hat{\tau}_c = \hat{\mu}_{1c} - \hat{\mu}_{0c}$
- Imbens and Kalyanaraman (2009) for bandwidth selection
- Can add regressors (not needed for identification, but can improve precision)
- According to Imbens and Lemieux (2008), adding regressors can help reduce bias from deterioration of the linear approximation.
- For limited dependent variable can use local versions of other estimators: Poisson, Logit, Probit, etc.
   - Example: $\hat{\tau}_c = exp(\hat{\alpha}_1 + \hat{\beta}_1c) - exp(\hat{\alpha}_0 + \hat{\beta}_0c)$



In [ ]:
%%stata

* Local regressions



## Fuzzy Regression Discontinuity Design (FRD)

### **Notation:**
- $P(x=1|x)\equiv F(x)$: propensity score as a function of $x$


### **Extra assumptions:**
- $F(\cdot)$ is discontinuous at $c$: discrete jump in prob. of treatment at $c$
- $y_1-y_0$: independent of $w$ given $x$

### **Identification:**

$$m(x) = \mu_0 + E[w|x]\cdot \tau(x) $$

Take limits to obtain: $m^+(c) = \mu_{0c} + F^+(c)\tau_c$ and $m^-(c) = \mu_{0c} + F^-(c)\tau_c$

If $F^+(c) \neq F^-(c)$, then 

$$\tau_c = \frac{m^+(c) - m^-(c)}{F^+(c) - F^-(c)}$$

### **Estimation:**
We can estimate $\tau_c$ by consistently estimating each of the quantities in the expresison above (e.g., by running four separate regressions, or using logit for the propensity scores):

$$ \hat\tau_c = \frac{\hat m^+(c) - \hat m^-(c)}{\hat F^+(c) - \hat F^-(c)}$$

For simple inference, use local IV estimator (restricting sample to a small window around $c$) using $z_i\equiv 1[x_i\geq c]$ as an instrument for $w_i$ in the following equation:

$$y_i = \alpha_{0c} + \tau_c w_i + \beta_0(x_i-c)+\delta 1[x_i\geq c]\cdot (x_i-c) + e_i$$


### **Tests**:

- Continuity of $\mu_{gc}$ may be violated if $x$ is manipulated (introduces selection).
- With no manipulation, we expect density of $x$ to be continuous at $c$.
- Visual check of histogram may be enough, but can also perform McCrary (2008) test of discontinuity in density of $x$ at $c$.